In [8]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from time import time
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.multioutput import ClassifierChain
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score
from sklearn import svm
import pandas as pd

In [9]:
#load the train and test data
DataTrain = pd.read_csv("train_kmedoids.csv") 
DataTest = pd.read_csv("test_kmedoids.csv")


In [10]:
DataTrain

,Cluster 0,Cluster 1,Cluster 2,Cluster 3,Cluster 4,Cluster 5,Cluster 6,Cluster 7,Cluster 8,Cluster 9,Cluster 10,Cluster 11,Cluster 12,Cluster 13,Cluster 14,Cluster 15,Cluster 16,Cluster 17,Cluster 18,Cluster 19
0,0.414548,0.401836,0.426836,0.411074,0.836155,0.544826,0.407771,0.408575,0.579547,0.944473,0.579162,0.391801,0.955807,0.412778,0.570592,0.427822,0.433890,0.831481,0.334216,0.536428
1,0.372133,0.328745,0.368366,0.374088,0.760516,0.490156,0.372669,0.238157,0.641974,0.850228,0.533825,0.307718,0.934594,0.364070,0.653746,0.366005,0.371403,0.431278,0.362992,0.490171
2,0.419354,0.413711,0.399528,0.423316,0.759667,0.488866,0.330713,0.396701,0.599053,0.858785,0.532925,0.411040,0.742596,0.424959,0.596838,0.336061,0.325223,0.605831,0.321356,0.496524
3,0.233521,0.344603,0.373293,0.185321,0.700270,0.524134,0.376851,0.319734,0.631277,0.814152,0.548552,0.308023,0.841190,0.307854,0.646235,0.301611,0.388868,0.835004,0.285463,0.511169
4,0.388933,0.371395,0.367590,0.433855,0.694607,0.490917,0.403075,0.410760,0.596351,0.799618,0.544618,0.259998,0.934694,0.416376,0.594559,0.377108,0.410364,0.798560,0.413026,0.501732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8246,0.392711,0.570641,0.341746,0.575189,0.699679,0.453218,0.411131,0.568737,0.386658,0.812642,0.406690,0.573911,0.930555,0.576597,0.338941,0.569699,0.573017,0.699894,0.390639,0.648460
8247,0.325950,0.306607,0.350355,0.332934,0.804760,0.491487,0.327824,0.311926,0.616904,0.915216,0.533120,0.339805,0.920657,0.317810,0.623657,0.363569,0.380629,0.802971,0.300249,0.482198
8248,0.391541,0.515390,0.408315,0.470186,0.727349,0.496488,0.338064,0.511802,0.375952,0.830690,0.451261,0.407656,0.921873,0.488018,0.289962,0.443136,0.399930,0.740799,0.387412,0.601816
8249,0.318722,0.277331,0.369482,0.276600,0.805092,0.501038,0.339816,0.293360,0.644031,0.915858,0.540153,0.307654,0.886827,0.310830,0.654251,0.370351,0.385523,0.799307,0.281402,0.325365


In [11]:
#load the labels 
train_label = [i.strip() for i in open("train-label.dat").readlines()]
test_label = [i.strip() for i in open("test-label.dat").readlines()]
print(len(train_label),len(test_label))

8251 3983


In [12]:
#shape the labels by label (each row represents one label) 
#for train data
labels_models=[]
for i in range (0,20 ):
    tr=[]
    for j in range (0,len(train_label)):
        tr.append(int(train_label[j][2*i]))
    labels_models.append(tr)
    
train_labels = pd.DataFrame(labels_models)
train_labels.shape    
#for test data
labels__models=[]
for i in range (0,20 ):
    ts=[]
    for j in range (0,len(test_label)):
        ts.append(int(test_label[j][2*i]))
    labels__models.append(ts)
    
test_labels = pd.DataFrame(labels__models)
test_labels.shape  

(20, 3983)

In [13]:
#find the transpose so each column represents one label
Labels_train = train_labels.transpose()
print("train data",Labels_train.shape)
Labels_train.head()

train data (8251, 20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,0,1,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0
4,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
Labels_test = test_labels.transpose()
print("train data",Labels_test.shape)
Labels_test.head()

train data (3983, 20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0


In [15]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
KNN = neighbors.KNeighborsClassifier(n_neighbors=3 ,weights='distance',metric='minkowski',p=1)

linear_svm = LinearSVC(random_state=0)
log_reg = LogisticRegression(random_state=0)    

In [16]:
Lab_tr = np.array(Labels_train)
Lab_ts = np.array(Labels_test)
Lab_tr

array([[1, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 1, ..., 1, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]], dtype=int64)

In [17]:
# Fit an ensemble of logistic regression classifier chains and
# take the average prediction of all the chains.
# each model gets the true labels as features
start_time = dt.datetime.now()

chains = [ClassifierChain(log_reg, order='random', random_state=i) for i in range(10)]
for chain in chains:
    chain.fit(DataTrain,Lab_tr)
finish_time = dt.datetime.now()
time = finish_time-start_time 
print('the training time is:',time)
Y_pred_chains = np.array([chain.predict(DataTest) for chain in chains])

Y_pred_ensemble_log =Y_pred_chains.mean(axis=0)


C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\MyPc\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this w

the training time is: 0:00:08.951832


In [18]:
len(Y_pred_chains)

10

In [19]:
from sklearn.metrics import confusion_matrix
def multilabel_confusion_matrix(predict):
    labels = []

    for item in open("labels.txt").readlines():
        item = item.replace('\n', '')
        label_name, _ = item.split(', ')
        labels.append(label_name)

    y_true = np.array(Lab_ts)
    y_pred = np.array(predict.round())

    conf_mat_dict = {}

    for label_col in range(len(labels)):
        y_true_label = y_true[:, label_col]
        y_pred_label = y_pred[:, label_col]
        conf_mat_dict[labels[label_col]] = confusion_matrix(y_pred=y_pred_label, y_true=y_true_label)

    for label, matrix in conf_mat_dict.items():
        print("Confusion matrix for label {}:".format(label))
        print(matrix)
    return  

In [20]:
#confusion matrix for Logistic Regression predictions
multilabel_confusion_matrix(Y_pred_ensemble_log)

Confusion matrix for label programming:
[[3006    0]
 [ 976    1]]
Confusion matrix for label style:
[[3755    0]
 [ 228    0]]
Confusion matrix for label reference:
[[2361   64]
 [1499   59]]
Confusion matrix for label java:
[[3611    0]
 [ 371    1]]
Confusion matrix for label web:
[[2933    0]
 [1050    0]]
Confusion matrix for label internet:
[[3446    0]
 [ 537    0]]
Confusion matrix for label culture:
[[3281    0]
 [ 702    0]]
Confusion matrix for label design:
[[2904    0]
 [1079    0]]
Confusion matrix for label education:
[[3180    0]
 [ 803    0]]
Confusion matrix for label language:
[[3500    0]
 [ 483    0]]
Confusion matrix for label books:
[[3476    0]
 [ 507    0]]
Confusion matrix for label writing:
[[3505    0]
 [ 478    0]]
Confusion matrix for label computer:
[[3474    0]
 [ 509    0]]
Confusion matrix for label english:
[[3628    0]
 [ 355    0]]
Confusion matrix for label politics:
[[3591    0]
 [ 392    0]]
Confusion matrix for label history:
[[3542    0]
 [ 441

In [21]:
# Fit an ensemble of SVM classifier chains and
# take the average prediction of all the chains.
# each model gets the true labels as features
start_time = dt.datetime.now()

chains = [ClassifierChain(linear_svm, order='random', random_state=i) for i in range(10)]
for chain in chains:
    chain.fit(DataTrain,Lab_tr)
finish_time = dt.datetime.now()
time = finish_time-start_time 
print('the training time is:',time)
Y_pred_chains = np.array([chain.predict(DataTest) for chain in chains])

Y_pred_ensemble_SVM =Y_pred_chains.mean(axis=0)




the training time is: 0:01:23.114158


In [22]:
multilabel_confusion_matrix(Y_pred_ensemble_SVM)

Confusion matrix for label programming:
[[2998    8]
 [ 972    5]]
Confusion matrix for label style:
[[3755    0]
 [ 228    0]]
Confusion matrix for label reference:
[[2360   65]
 [1493   65]]
Confusion matrix for label java:
[[3607    4]
 [ 372    0]]
Confusion matrix for label web:
[[2933    0]
 [1050    0]]
Confusion matrix for label internet:
[[3446    0]
 [ 537    0]]
Confusion matrix for label culture:
[[3281    0]
 [ 702    0]]
Confusion matrix for label design:
[[2904    0]
 [1079    0]]
Confusion matrix for label education:
[[3180    0]
 [ 803    0]]
Confusion matrix for label language:
[[3500    0]
 [ 483    0]]
Confusion matrix for label books:
[[3476    0]
 [ 507    0]]
Confusion matrix for label writing:
[[3505    0]
 [ 478    0]]
Confusion matrix for label computer:
[[3474    0]
 [ 509    0]]
Confusion matrix for label english:
[[3628    0]
 [ 355    0]]
Confusion matrix for label politics:
[[3591    0]
 [ 392    0]]
Confusion matrix for label history:
[[3542    0]
 [ 441

In [23]:
# Fit an ensemble of KNN classifier chains and
# take the average prediction of all the chains.
# each model gets the true labels as features
start_time = dt.datetime.now()

chains = [ClassifierChain(KNN, order='random', random_state=i) for i in range(10)]
for chain in chains:
    chain.fit(DataTrain,Lab_tr)
finish_time = dt.datetime.now()
time = finish_time-start_time 
print('the training time is:',time)
start_time = dt.datetime.now()
Y_pred_chains = np.array([chain.predict(DataTest) for chain in chains])

Y_pred_ensemble_KNN =Y_pred_chains.mean(axis=0)

finish_time = dt.datetime.now()
time = finish_time-start_time 
print('the prediction time is:',time)


the training time is: 0:00:04.498423


In [24]:
multilabel_confusion_matrix(Y_pred_ensemble_KNN)

Confusion matrix for label programming:
[[2713  293]
 [ 795  182]]
Confusion matrix for label style:
[[3720   35]
 [ 216   12]]
Confusion matrix for label reference:
[[1888  537]
 [1134  424]]
Confusion matrix for label java:
[[3533   78]
 [ 344   28]]
Confusion matrix for label web:
[[2569  364]
 [ 815  235]]
Confusion matrix for label internet:
[[3335  111]
 [ 517   20]]
Confusion matrix for label culture:
[[3118  163]
 [ 621   81]]
Confusion matrix for label design:
[[2603  301]
 [ 942  137]]
Confusion matrix for label education:
[[3063  117]
 [ 759   44]]
Confusion matrix for label language:
[[3457   43]
 [ 459   24]]
Confusion matrix for label books:
[[3426   50]
 [ 486   21]]
Confusion matrix for label writing:
[[3431   74]
 [ 457   21]]
Confusion matrix for label computer:
[[3411   63]
 [ 493   16]]
Confusion matrix for label english:
[[3582   46]
 [ 338   17]]
Confusion matrix for label politics:
[[3507   84]
 [ 365   27]]
Confusion matrix for label history:
[[3498   44]
 [ 423